# Pydantic

## 1. Parsing Strings
Pydantic은 문자열 변환을 위한 올바른 유형을 알아냅니다. 처음 사용했을 때는 데이터를 수동으로 변환한 후 Pydantic 모델에 전달하기 위해 여러 가지 이상한 방법을 시도했습니다. 
Pydantic은 이를 자동으로 처리하여 많은 수고를 덜어주었습니다.다음은 그 예입니다:

In [1]:
from pydantic import BaseModel
from datetime import date

class MyModel(BaseModel):
    integer_field: int
    float_field: float
    date_field: date

data = {
    "integer_field": "42",
    "float_field": "3.1415",
    "date_field": "2021-12-31"
}

model = MyModel(**data)
print(model)
# integer_field=42 float_field=3.1415 date_field=datetime.date(2021, 12, 31)
print(type(model.integer_field))
# <class 'int'>
print(type(model.float_field))
# <class 'float'>
print(type(model.date_field))
# <class 'datetime.date'>

integer_field=42 float_field=3.1415 date_field=datetime.date(2021, 12, 31)
<class 'int'>
<class 'float'>
<class 'datetime.date'>


## 2. Generating JSON Schemas
저는 LLM을 많이 사용하는데, 도구 정의와 구조화된 출력을 위해서는 JSON 스키마를 정의해야 했습니다. 처음에는 수동으로 정의했지만, Pydantic 모델을 생성하고 이를 통해 스키마를 쉽게 생성할 수 있다는 사실을 알게 되었습니다. 다음 스니펫에서 볼 수 있듯이 매우 간단합니다:

In [2]:
from pydantic import BaseModel
from typing import List, Optional
import json

class Parameter(BaseModel):
    name: str
    type: str
    required: bool = True

class Tool(BaseModel):
    tool_name: str
    version: Optional[str] = None
    parameters: List[Parameter]

# Generate JSON schema
schema_dict = Tool.model_json_schema()
schema_json = json.dumps(schema_dict, indent=2)
print(schema_json)

{
  "$defs": {
    "Parameter": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "type": {
          "title": "Type",
          "type": "string"
        },
        "required": {
          "default": true,
          "title": "Required",
          "type": "boolean"
        }
      },
      "required": [
        "name",
        "type"
      ],
      "title": "Parameter",
      "type": "object"
    }
  },
  "properties": {
    "tool_name": {
      "title": "Tool Name",
      "type": "string"
    },
    "version": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Version"
    },
    "parameters": {
      "items": {
        "$ref": "#/$defs/Parameter"
      },
      "title": "Parameters",
      "type": "array"
    }
  },
  "required": [
    "tool_name",
    "parameters"
  ],
  "title": "Tool",
  "type": "object"
}


## 3. Partial JSON Parsing
이는 LLM에서 JSON 응답을 스트리밍할 때 출력의 유효성을 검사하거나 구문 분석해야 할 때 매우 유용합니다. 전체 응답을 기다리는 대신 부분 데이터를 구문 분석하여 유효한 결과를 얻을 수 있습니다. 다음은 JSON 문자열의 부분 구문 분석을 허용하는 간단한 예제입니다.

In [5]:
partial_json_data = '["aa", "bb", "c'

result = from_json(partial_json_data, allow_partial=True)
print(result)
# ['aa', 'bb']

NameError: name 'from_json' is not defined

## 4. Partial Pydantic Model

이 기능은 LLM 작업에 매우 유용하며 부분 JSON 구문 분석과 잘 작동합니다. Pydantic 모델의 필드를 선택 사항으로 만들 수 있으므로 나머지 데이터가 스트리밍되기를 기다리는 동안 JSON을 부분적으로 파싱하여 모델의 인스턴스로 변환할 수 있습니다.저는 Pydantic 모델의 모든 필드를 선택 사항으로 만드는 이 '트릭'을 생각해낸 adriangb로부터 이 방법을 배웠습니다. 예제에서 볼 수 있듯이 오류 없이 하나의 필드만 전달할 수 있습니다. 스트리밍 컨텍스트에서는 워크플로우에서 이미 구문 분석된 필드를 사용하여 지연 시간을 줄일 수 있습니다. 이에 대한 자세한 내용은 곧 블로그를 작성할 예정이니 기대해 주세요.

In [6]:
from copy import deepcopy
from typing import Any, Optional, Tuple, Type, TypeVar

from pydantic import BaseModel, create_model
from pydantic.fields import FieldInfo

class User(BaseModel):
  first_name: str
  last_name: str

def make_field_optional(field: FieldInfo, default: Any = None) -> Tuple[Any, FieldInfo]:
  new = deepcopy(field)
  new.default = default
  new.annotation = Optional[field.annotation]  # type: ignore
  return (new.annotation, new)

BaseModelT = TypeVar('BaseModelT', bound=BaseModel)

def make_partial_model(model: Type[BaseModelT]) -> Type[BaseModelT]:
  return create_model(  # type: ignore
    f'Partial{model.__name__}',
    __base__=User,
    __module__=User.__module__,
    **{
        field_name: make_field_optional(field_info)
        for field_name, field_info in User.model_fields.items()
    }
    )

PartialUser = make_partial_model(User)

print(PartialUser(first_name='Adrian'))
#> first_name='Adrian' last_name=None

first_name='Adrian' last_name=None


## 5. Add Extra Configuration to Your Schemas
LLM 에이전트에서 이 기능이 구현된 것을 봤는데, 기능 2번의 확장판과 비슷합니다. 기본적으로 Pydantic 모델의 JSON 스키마를 확장하여 더 많은 정보를 제공할 수 있습니다. 이는 짧은 예시를 LLM에 전달하려는 경우에 유용합니다. 긴 프롬프트를 만드는 대신 스키마에 추가하기만 하면 자동으로 LLM에 전달됩니다.방법은 다음과 같습니다:

In [15]:
from pydantic import BaseModel, ConfigDict
import json

class QAEntry(BaseModel):
    question: str
    answer: str

    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "question": "파이썬 `filter()` 함수는 어떻게 작동하나요?",
                    "answer": "`filter()` 함수는 함수가 참을 반환하는 이터러블의 요소로부터 이터레이터를 구성합니다."
                },
                {
                    "question": "객체 지향 프로그래밍에서 polymorphism이란 무엇인가요?",
                    "answer": "polymorphism은 서로 다른 클래스의 객체를 공통 슈퍼 클래스의 객체로 취급하여 단일 인터페이스가 서로 다른 기본 형태를 나타낼 수 있도록합니다."
                },
                {
                    "question": "재귀의 개념을 설명해 주시겠어요?",
                    "answer": "재귀는 동일한 문제의 작은 인스턴스를 해결하기 위해 함수가 스스로를 호출하는 프로그래밍 기법입니다."
                }
            ]
        }
    )

# 영어 키를 한글 키로 바꾸는 매핑
key_translation = {
    "title": "제목",
    "type": "유형",
    "properties": "속성",
    "question": "질문",
    "answer": "답변",
    "examples": "예시",
    "required": "필수항목",
    "type": "유형",
}

# 재귀적으로 키 이름을 한글로 변환
def translate_keys(obj):
    if isinstance(obj, dict):
        return {
            key_translation.get(k, k): translate_keys(v)
            for k, v in obj.items()
        }
    elif isinstance(obj, list):
        return [translate_keys(i) for i in obj]
    else:
        return obj

# 원본 스키마 → 키 한글 변환 → JSON 문자열 출력
schema_dict = QAEntry.model_json_schema()
translated_schema = translate_keys(schema_dict)
schema_json = json.dumps(translated_schema, indent=2, ensure_ascii=False)
print(schema_json)

{
  "예시": [
    {
      "답변": "`filter()` 함수는 함수가 참을 반환하는 이터러블의 요소로부터 이터레이터를 구성합니다.",
      "질문": "파이썬 `filter()` 함수는 어떻게 작동하나요?"
    },
    {
      "답변": "polymorphism은 서로 다른 클래스의 객체를 공통 슈퍼 클래스의 객체로 취급하여 단일 인터페이스가 서로 다른 기본 형태를 나타낼 수 있도록합니다.",
      "질문": "객체 지향 프로그래밍에서 polymorphism이란 무엇인가요?"
    },
    {
      "답변": "재귀는 동일한 문제의 작은 인스턴스를 해결하기 위해 함수가 스스로를 호출하는 프로그래밍 기법입니다.",
      "질문": "재귀의 개념을 설명해 주시겠어요?"
    }
  ],
  "속성": {
    "질문": {
      "제목": "Question",
      "유형": "string"
    },
    "답변": {
      "제목": "Answer",
      "유형": "string"
    }
  },
  "필수항목": [
    "question",
    "answer"
  ],
  "제목": "QAEntry",
  "유형": "object"
}
